In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
csvpath = '../../LHC_mimic/mimic3_1.4/derived/notes.words.csv'
# csvpath = '/Z/'

In [3]:
df = pd.read_csv(csvpath, names = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'CATEGORY', 'DESCRIPTION', 'OBSERVATIONS'])

In [4]:
df['OBSERVATIONS'] = [obs.split(' ') for obs in df['OBSERVATIONS'] ]

In [5]:
df

,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,OBSERVATIONS
0,94125,187893.0,2188-08-07,2188-08-07T05:29,Nursing,Nursing Progress Note,"[Ventilation, , Impaired, , , , Assessment, , ..."
1,26127,161400.0,2147-11-10,2147-11-10T17:28,Nursing/other,Report,"[Neonatology, Attending, DOL, 12, , , **Known,..."
2,2543,106163.0,2114-09-29,2114-09-29T06:14,Nursing/other,Report,"[Respiratory, Care, , Patient, remains, on, ve..."
3,631,100660.0,2124-01-25,2124-01-25T18:59,Nursing/other,Report,"[Nursing, Note, 7a-7p, S, , ""They, said, I'm, ..."
4,70963,110511.0,2107-03-03,2107-03-03T03:58,Radiology,CHEST (PORTABLE AP),"[, **2107-3-3**, , 3, 58, AM, , CHEST, , PORTA..."
...,...,...,...,...,...,...,...
2048819,26512,182452.0,2122-11-07,2122-11-07T16:06,Nursing/other,Report,"[, **Name, , NI, , 1021**, , Care, Pt, recieve..."
2048820,96864,NaN,2103-01-12,NaN,Echo,Report,"[PATIENT/TEST, INFORMATION, , Indication, , pa..."
2048821,19307,151814.0,2110-06-06,2110-06-06T04:27,Nursing/other,Report,"[Nursing, , , #2O, , , Temp, stable, , swaddle..."
2048822,5106,161889.0,2172-11-18,2172-11-18T16:16,Nursing/other,Report,"[Nursing, Progress, Note, , , RESP, O/A, , Inf..."


In [6]:
def add_freqs(lst, freqdict):
    for concept in lst:
        if concept in freqdict:
            freqdict[concept]+=1
        else:
            freqdict[concept]=1

In [7]:
vocab_freqs = {}
df['OBSERVATIONS'].apply(lambda x: add_freqs(x, vocab_freqs))

0          None
1          None
2          None
3          None
4          None
           ... 
2048819    None
2048820    None
2048821    None
2048822    None
2048823    None
Name: OBSERVATIONS, Length: 2048824, dtype: object

In [8]:
vocab = sorted(((value, key) for (key,value) in vocab_freqs.items()), reverse=True)


In [9]:
sorted_ids = {tup[1]:i+1 for i,tup in enumerate(vocab)}

In [10]:
sorted_ids['Nursing']

948

In [11]:
import json
with open('words_sorted_id.json','w') as jsonfile:
    json.dump(sorted_ids, jsonfile)

In [12]:
len(list(vocab_freqs.keys()))

1891434

In [22]:
id_dict = {k:v for v,k in enumerate(list(vocab_freqs.keys())) }

In [24]:
import json
with open('cui_vocab_id.json','w') as jsonfile:
    json.dump(id_dict, jsonfile)

In [2]:
##CREATE SPARSE TENSOR USING CUI VOCAB IDS##
import json
import pandas as pd
# import tensorflow as tf

In [3]:
import os

In [4]:
vocab_path = 'cui_vocab_id.json'
with open(vocab_path, 'r') as vocfile:
    vocab = json.load(vocfile)

In [5]:
keylist = list(vocab.keys())
dense_cols = len(keylist)

In [6]:
task = 'in-hospital-mortality'
train_path = './data/just_cui_notes/validated/'+task+'/train/'
embed_path = ''

In [7]:
for filename in os.listdir(train_path)[0:2]:
    if filename == 'listfile.csv':
        continue
    filepath = train_path + filename
    df = pd.read_csv(filepath)[['Hours', 'TEXT']]
#     df['TEXT'] =  [list(set(obs.split(' '))) for obs in df.TEXT]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(vocabulary=[word.lower() for word in keylist], binary=True)

In [69]:
spc = pd.DataFrame.sparse.from_spmatrix(cv.fit_transform(df['TEXT']))

In [71]:
# filter = (spc != 0).any()
# spc.loc[:, filter]
spc.insert(0,'Hours',df.Hours)

In [10]:
nparr = cv.fit_transform(df['TEXT'])

In [11]:
nparr.todense()

matrix([[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [12]:
from scipy.sparse import hstack
import numpy as np
spc = hstack((np.array(df['Hours'])[:,None],nparr))

In [13]:
spc = [row for row in spc]

TypeError: 'coo_matrix' object is not subscriptable

In [100]:
import scipy
scipy.sparse.save_npz('test_sparse_matrix.npz', spc)


In [101]:
sparse_matrix = scipy.sparse.load_npz('test_sparse_matrix.npz')

In [102]:
sparse_matrix.todense()

matrix([[ 1.90138889,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [12.50138889,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [23.05138889,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [73.31805556,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [74.21805556,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [82.68472222,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

In [1]:
###TEST TF SPARSE
import tensorflow as tf
import numpy as np

example = tf.SparseTensor(indices=[[0], [1], [2]], values=[3, 6, 9], dense_shape=[3])

vocabulary_size = 10
embedding_size = 1
var = np.array([0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0])
embeddings = tf.Variable(var)

embed = tf.nn.embedding_lookup_sparse(embeddings, example, None)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(sess.run(embed)) # prints [  9.  36.  81.]

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[ 9. 36. 81.]


In [2]:
print(example)

SparseTensor(indices=Tensor("SparseTensor/indices:0", shape=(3, 1), dtype=int64), values=Tensor("SparseTensor/values:0", shape=(3,), dtype=int32), dense_shape=Tensor("SparseTensor/dense_shape:0", shape=(1,), dtype=int64))
